In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%gui qt
import napari
from skimage import io
import seaborn as sns
from nd2reader import ND2Reader
from scipy.fftpack import dct

In [24]:
%matplotlib qt

In [2]:
os.chdir(r'Z:\TAD\200610_adipogenesis_timecourse_r10_v2\gr-rxr-set')

In [11]:
os.listdir()

['dapi0.tif',
 'dapi0_segm_.tiff',
 'dapi1.tif',
 'dapi1_segm_.tiff',
 'dapi2.tif',
 'dapi2_segm_.tiff',
 'stad424',
 'stad425_rois002.nd2',
 'stad425_rois003.nd2',
 'stad425_rois004.nd2',
 'stad430_rois002.nd2',
 'temp_imgs',
 'z_max.tif',
 'z_max_.tif',
 'z_mean.tif',
 'z_mean_.tif']

In [14]:
fl_multipoints = ND2Reader('stad425_rois004.nd2')

In [12]:
fl_multipoints.bundle_axes = 'czyx'
fl_multipoints.iter_axes = 'v'

In [10]:
dic_multipoints = ND2Reader('stad424_test_dic001.nd2')

FileNotFoundError: [Errno 2] No such file or directory: 'stad424_test_dic001.nd2'

In [14]:
dic_multipoints.bundle_axes = 'czyx'
dic_multipoints.iter_axes = 'v'

In [15]:
fl_multi_arr = np.array(fl_multipoints).astype('uint16')
dic_multi_arr = np.array(dic_multipoints).astype('uint16')

C:\Users\LegantLab\anaconda3\lib\site-packages\nd2reader\parser.py:292: UserWarning: ND2 file contains gap frames which are represented by np.nan-filled arrays; to convert to zeros use e.g. np.nan_to_num(array)
  warnings.warn('ND2 file contains gap frames which are represented by np.nan-filled arrays; to convert to zeros use e.g. np.nan_to_num(array)')


In [20]:
viewer = napari.Viewer()
viewer.add_image(fl_multi_arr[0, :, :, :, :])
viewer.add_image(dic_multi_arr[0, :, :, :, :])

<Image layer 'Image [1]' at 0x1a881c79c08>

In [17]:
fl_multi_arr.shape

(3, 4, 45, 2044, 2048)

In [60]:
fig, axes = plt.subplots(2,2)
axes = axes.flatten()

axes[0].imshow(fl_multi_arr[1, 0, 23, :, :])
axes[1].imshow(fl_multi_arr[1, 1, 23, :, :])
axes[2].imshow(fl_multi_arr[1, 2, 23, :, :])
axes[3].imshow(fl_multi_arr[1, 3, 23, :, :])

In [40]:
'''This will calculate and return the shannon entropy of the normalized discrete cosine transform. This is 
taken from strategy used in doi: 10.1038/nbt.3708. takes in a 2D array and spits out a scaler, the dcst.
'''
def dcts(img_):
    dis_cos = dct(img_) #take the discrete cosine transform of the image
    l2 = np.sqrt(np.sum(np.square(img_))) #perform l2 normalization
    inner_term = np.divide(dis_cos, l2) #inner term of the shannon enropy
    '''need to get rid of any zeros that occur in inner term'''
    inner_term[inner_term == 0] = .0001
    first_term = np.abs(inner_term) #first term of shannon entropy
    second_term = np.log2(np.abs(inner_term)) #second term of shannon entropy 
    return(np.multiply(-1, np.sum(np.multiply(first_term, second_term))))
    #return(second_term)

In [27]:
'''This will calculate and return the shannon entropy of the normalized discrete cosine transform. This is 
taken from strategy used in doi: 10.1038/nbt.3708. takes in a 2D array and spits out a scaler, the dcst.
'''
def dcts(img_):
    dis_cos = dct(img_) #take the discrete cosine transform of the image
    l2 = np.sqrt(np.sum(np.square(img_))) #perform l2 normalization
    inner_term = np.divide(dis_cos, l2) #inner term of the shannon enropy
    '''need to get rid of any zeros that occur in inner term'''
    inner_term[inner_term == 0] = .0001
    first_term = np.abs(inner_term) #first term of shannon entropy
    second_term = np.log2(np.abs(inner_term)) #second term of shannon entropy 
    return(np.multiply(-1, np.sum(np.multiply(first_term, second_term))))
    #return(second_term)

In [67]:
img_number = 0

for img_number in range(fl_multi_arr.shape[0]):
    dcts_lst = []
    
    dapi_img = fl_multi_arr[img_number, 3, :, :, :]
    for zslice in range(dapi_img.shape[0]):
        dapi_slice = dapi_img[zslice, :, :]
        dcts_lst.append(dcts(dapi_slice))
    min_entropy = np.argwhere(np.array(dcts_lst) == np.min(dcts_lst))
    io.imsave('dapi' + str(img_number) + '.tif', dapi_img[min_entropy, :, :])
    
    



In [ ]:
c1_img = fl_multi_arr[img_number, 0, :, :, :]
c2_img = fl_multi_arr[img_number, 1, :, :, :]
c3_img = fl_multi_arr[img_number, 2, :, :, :]
c4_img = fl_multi_arr[img_number, 3, :, :, :]

c1_dcts_lst = []
c2_dcts_lst = []
c3_dcts_lst = []
c4_dcts_lst = []

for i in range(c1.shape[0]):
    c1_dcts_lst.append(dcts(c1_img[i, :, :]))
    c2_dcts_lst.append(dcts(c2_img[i, :, :]))
    c3_dcts_lst.append(dcts(c3_img[i, :, :]))
    c4_dcts_lst.append(dcts(c4_img[i, :, :]))

In [54]:
fl_multi_arr.shape

(3, 4, 45, 2044, 2048)

In [43]:
dcts_lst_lst = [c1_dcts_lst, c2_dcts_lst, c3_dcts_lst, c4_dcts_lst]

for i in range(len(dcts_lst_lst)):
    dcts_lst_ = dcts_lst_lst[i]
    ax = plt.plot(list(range(len(dcts_lst_))), dcts_lst_)

In [53]:
plt.plot(list(range(len(dcts_lst_))), c4_dcts_lst)

In [37]:
os.listdir()

['stad424',
 'stad424_test.nd2',
 'stad424_test_dic.nd2',
 'stad424_test_dic001.nd2',
 'test.tif']

In [4]:
sing_fov = ND2Reader('stad425_rois003.nd2')
sing_fov.bundle_axes = 'zyx' #bundle the channel and zyx coordinates
sing_fov.iter_axes = 'c' # set the index to the field of view

jnk = np.array(sing_fov, dtype = 'uint16')

In [5]:
jnk.shape

(4, 25, 2044, 2048)

In [6]:
sing_fov = ND2Reader('stad425_rois003.nd2')

In [7]:
sing_fov.sizes

{'x': 2048, 'y': 2044, 'c': 4, 't': 1, 'z': 25}

In [15]:
fl_multipoints.sizes

{'x': 2048, 'y': 2044, 'c': 4, 't': 1, 'z': 25, 'v': 5}

In [17]:
lst = [sing_fov, fl_multipoints]

for i in lst:
    if 'v' not in i.sizes:
        print('yes')
    else:
        print('no')

yes
no


In [22]:
jnk = list(fl_multipoints.sizes.keys())

In [23]:
type(jnk)

list

In [24]:
jnk

['x', 'y', 'c', 't', 'z', 'v']

In [25]:
import napari

In [26]:
viewer = napari.Viewer()
viewer.add_image(fl_multipoints)

AttributeError: 'ND2Reader' object has no attribute 'shape'

In [28]:
fl_multipoints.bundle_axes = 'czyx'
fl_multipoints.iter_axes = 'v'

In [29]:
jnk = fl_multipoints[0]

In [30]:
jnk.shape

(4, 25, 2044, 2048)

In [31]:
type(jnk)

pims.frame.Frame

In [33]:
fl_multipoints.sizes

{'x': 2048, 'y': 2044, 'c': 4, 't': 1, 'z': 25, 'v': 5}